# Data Wrangling Cheatsheet Using Pandas - Multiple Ways to Select Columns and rows in Pandas

# Data Wrangling Cheatsheet Using Pandas

### A problem with using pandas  is that there are multiple ways to achieve a given result. Hence googling can introduce a random variability around the best way to achieve a result. Sometimes you hit a particular result and the next time for a very  similar query you get an entirely different way of doing it. For the newbie coder this can be confusing and also lead to the feeling that coding is a mysterious process which can only be googled!

### In this notebook I am looking to provide multiple ways in which columns and rows can be selected since this is a key step in data wrangling both for data exploration and also for feature engineering. This is a cheatsheet which I prepared for myself after bumbling around with different methods and not making much progress in understanding why something is used in a particular case and not in other seemingly similar scenarios.



# Challenges of data wrangling in Pandas

### Real world data wrangling can be challenging for those of us who are non coders. A common scenario after importing a data set is to stare at it blankly trying to recall what a particular code/syntax is for doing tasks which would take precisely a minute in Excel. Sometimes the tempatation is to finsh the entire data preperation phase in Excel and then import a clean data set to a Jupyter Notebook. Unfortunately this is only viable when the data set is small or moderate. Plus as a newbie coder it makes sense to do as many tasks as possible via coding.

### I  then come to the second challenge of coding: googling the answer. Now, Google and Stackoverflow are great and frequently do provide the answers. But there is problem. Your coding skills will not evolve as one tends to 'copy paste' the code snippet without thinking about it at lenght. 

### Row and column selection is a very basic process in data exploration. It's a step one wants to get done quickly and then move onto more advanced stuff. There are multiple ways of doing this in Pandas and Python. It can be tricky figuring what is the best method for a particular query. In this notebook I will create a toy data set and then discuss some common data wrangling problems and the best fix coding solution.

### So let's get to it!

### Setting up the data and the problem. I am going to create a toy data set primarily for ease of illustrating specific analysis use cases

In [16]:
import numpy as np
import pandas as pd

# Creating the dataframe

### I am looking to create a simple Pandas dataframe which has a text column along with a few quantitative data columns. An interesting business use case is customer level purchase data along with VOC score as well as some other quantitative metrics such as number of calls made to a customer. At the customer level we also have some demographic data such as gender, and age. In terms of text variables we have an open end verbatim response on 'Suggestions' by the customers. Now this is a toy set but it encapsulates a familiar set of business problems. Key business objectives of this analysis could be to undertand VOC responses in terms of satisfaction and whether it is driven by purchase behaviour or demographic  characteristics of the  customer.


In [17]:
df = pd.DataFrame({"cust_name":["Jeff","sejal","miff","seb","deb","fema","diva", "may"], "VOC":[6,7,8,9,10,3,4,5], "purchase":[23,42,15,67,56,78,12,25], "calls":[1,2,1,3,2,0,1,4], "suggestions": ["the product needs improvement","the taste is too sharp and it is chewy", "I loved it and will buy regularly","It has  nice flavours and varieties", "this is a great product, at a good price", "It does not have a good taste and high price", "I will never buy again as it is too costly", "it is too expensive " ], "age":[23,15,34,45,19,26,33,15], "region":["north","north","south","east","south","north","west","north"],"gender":["male","male","male","female","female","male","male","female"]})

In [18]:
#checking the dataframe
df.head()

,cust_name,VOC,purchase,calls,suggestions,age,region,gender
0,Jeff,6,23,1,the product needs improvement,23,north,male
1,sejal,7,42,2,the taste is too sharp and it is chewy,15,north,male
2,miff,8,15,1,I loved it and will buy regularly,34,south,male
3,seb,9,67,3,It has nice flavours and varieties,45,east,female
4,deb,10,56,2,"this is a great product, at a good price",19,south,female


## Setting up the problem. This is a small data set but it encapsulates in a toy form several standard EDA possibilities. We would like to explore for example:
## 1. Is there a relationship between VOC score and various usage and demographic categories (age, purchase, calls,gender and region). It doesn't make sense to run these with raw scores hence somekind of bucketing needs to be done
## 2. It would be good to check if the product or brand is being mentioned in the verbatim and whether there are any brand related key words
## All these are terribly simple things to do in Excel, but with Python and Panda's it requires a little more knowledge. First up to analyze the VOC we can caategorize it into 'promoters: 9-10', 'Neutral: 7-8' and 'Detractors:0-6'.
### Now there are various ways to do this bucketing Pandas and Python. I will demonstrate the ones I feel are the most userfriendly and flexible and intuitive. By intuitive I mean that the code is understandable and short.

# Bucketing method 1: Identifying rows based on a condition and using Pandas .loc() and storing result in a separate column. 
### The loc is label-based, which means that we have to specify the name of the rows and columns that we need to filter out.
###  syntax: df.loc[[df["col"] condition, "new col"].
### Notice that we do the following here: 1. we can use a condition applied on rows to filter out rows that belong to Promoters. We are also able to assign these rows the appropriate lable in NPS_status column. Secondly we are able to use a multiple if conditions to determine Passives.

In [19]:
df.loc[df['VOC']>=9, "NPS_status"]="Promoters"
df.loc[(df['VOC']>6) & (df['VOC']<9), "NPS_status"]="Passives"
df.loc[df['VOC']<7, "NPS_status"]="Detractors"


In [20]:
df

,cust_name,VOC,purchase,calls,suggestions,age,region,gender,NPS_status
0,Jeff,6,23,1,the product needs improvement,23,north,male,Detractors
1,sejal,7,42,2,the taste is too sharp and it is chewy,15,north,male,Passives
2,miff,8,15,1,I loved it and will buy regularly,34,south,male,Passives
3,seb,9,67,3,It has nice flavours and varieties,45,east,female,Promoters
4,deb,10,56,2,"this is a great product, at a good price",19,south,female,Promoters
5,fema,3,78,0,It does not have a good taste and high price,26,north,male,Detractors
6,diva,4,12,1,I will never buy again as it is too costly,33,west,male,Detractors
7,may,5,25,4,it is too expensive,15,north,female,Detractors


# Method 2: Using Lambda functions
### Interestingly and what makes things confusing if you are googling this is not the only way to get this result. Method 2 makes use of lambda functions and the apply() function. I found this as  one of the methods that pops up when you google the query, 'Pandas + checking rows for if condition and assigning a string variable in another column'.
### syntax: df['newcol']=df['col'].apply(lambda x: 'value of if conition met' if condition' else 'value of condition not met')
### Unfortunately lambda can be tricky with multiple 'if' conditions, however as you can see I got the below code to work. I think this is a second best option because of challenges with more complex 'if' conditions which the loc method handles better.

In [21]:
#Let's create another col NPS2_status
df['NPS2_status']= df['VOC'].apply(lambda x: "Promoters" if x >=9 else( "Detractors" if x<7 else "Passives"))


In [22]:
df

,cust_name,VOC,purchase,calls,suggestions,age,region,gender,NPS_status,NPS2_status
0,Jeff,6,23,1,the product needs improvement,23,north,male,Detractors,Detractors
1,sejal,7,42,2,the taste is too sharp and it is chewy,15,north,male,Passives,Passives
2,miff,8,15,1,I loved it and will buy regularly,34,south,male,Passives,Passives
3,seb,9,67,3,It has nice flavours and varieties,45,east,female,Promoters,Promoters
4,deb,10,56,2,"this is a great product, at a good price",19,south,female,Promoters,Promoters
5,fema,3,78,0,It does not have a good taste and high price,26,north,male,Detractors,Detractors
6,diva,4,12,1,I will never buy again as it is too costly,33,west,male,Detractors,Detractors
7,may,5,25,4,it is too expensive,15,north,female,Detractors,Detractors


# Method 3: use numpy where() function. 
### I came across this method in one my googles.It is fast but has the con that we the output is a boolean object. Hence you need to do this for two way categorizations, then do a further transfer to the string labels.The output from the np.where, which is a list of row index matching the multiple conditions is fed to dataframe loc function.
### So for VOC categorization using numpy "where" can only be used for a binary classification. So, for example in our VOC classification we can classify into "Promoters" and "Others"


In [23]:
df['NPS_st_np']= np.where(df["VOC"]>=9, "Promoters", "Others")

In [24]:
df

,cust_name,VOC,purchase,calls,suggestions,age,region,gender,NPS_status,NPS2_status,NPS_st_np
0,Jeff,6,23,1,the product needs improvement,23,north,male,Detractors,Detractors,Others
1,sejal,7,42,2,the taste is too sharp and it is chewy,15,north,male,Passives,Passives,Others
2,miff,8,15,1,I loved it and will buy regularly,34,south,male,Passives,Passives,Others
3,seb,9,67,3,It has nice flavours and varieties,45,east,female,Promoters,Promoters,Promoters
4,deb,10,56,2,"this is a great product, at a good price",19,south,female,Promoters,Promoters,Promoters
5,fema,3,78,0,It does not have a good taste and high price,26,north,male,Detractors,Detractors,Others
6,diva,4,12,1,I will never buy again as it is too costly,33,west,male,Detractors,Detractors,Others
7,may,5,25,4,it is too expensive,15,north,female,Detractors,Detractors,Others


# Checking on verbatim columns
### Another important thing that is important is analysis of a column of verbatim. Typically one basic thing we need to do is to check for whether the verbatim contains a keyword of interest. There are a couple of ways we can do this. I present 2 methods below.
### 1. Using a lambda function and 'in'
### 2. Using a series.str.contains() function to test is a particular pattern or regex is contained within a series.

## Method 1 using lambda function
### Suppose we want to check how many of the verbatim are referring to price. Now one thing is checking for price. We may also want to check for synonyms such as expensive or costly. this is more complex and lamda is not the best way of doing this.
### Using lambda is a bit more rigid and we can easily check for only one keyword

df['Price']=df["suggestions"].apply(lambda x: 1 if "price" in x else 0)

In [25]:
df

,cust_name,VOC,purchase,calls,suggestions,age,region,gender,NPS_status,NPS2_status,NPS_st_np
0,Jeff,6,23,1,the product needs improvement,23,north,male,Detractors,Detractors,Others
1,sejal,7,42,2,the taste is too sharp and it is chewy,15,north,male,Passives,Passives,Others
2,miff,8,15,1,I loved it and will buy regularly,34,south,male,Passives,Passives,Others
3,seb,9,67,3,It has nice flavours and varieties,45,east,female,Promoters,Promoters,Promoters
4,deb,10,56,2,"this is a great product, at a good price",19,south,female,Promoters,Promoters,Promoters
5,fema,3,78,0,It does not have a good taste and high price,26,north,male,Detractors,Detractors,Others
6,diva,4,12,1,I will never buy again as it is too costly,33,west,male,Detractors,Detractors,Others
7,may,5,25,4,it is too expensive,15,north,female,Detractors,Detractors,Others


# Method 2: using series.str.contains()
### Whether a particular regex is contained in the string. This returns a boolean series based on whether it is contained.
### This then requires to be converted to a numeric value, for example, for further processing using for example lambda or .loc functions.

In [27]:
# Suppose we want to pull out all the rows with verbatim related to price
df["price_1"]=df["suggestions"].str.contains("price|costly|expensive")

In [28]:
df

,cust_name,VOC,purchase,calls,suggestions,age,region,gender,NPS_status,NPS2_status,NPS_st_np,price_1
0,Jeff,6,23,1,the product needs improvement,23,north,male,Detractors,Detractors,Others,False
1,sejal,7,42,2,the taste is too sharp and it is chewy,15,north,male,Passives,Passives,Others,False
2,miff,8,15,1,I loved it and will buy regularly,34,south,male,Passives,Passives,Others,False
3,seb,9,67,3,It has nice flavours and varieties,45,east,female,Promoters,Promoters,Promoters,False
4,deb,10,56,2,"this is a great product, at a good price",19,south,female,Promoters,Promoters,Promoters,True
5,fema,3,78,0,It does not have a good taste and high price,26,north,male,Detractors,Detractors,Others,True
6,diva,4,12,1,I will never buy again as it is too costly,33,west,male,Detractors,Detractors,Others,True
7,may,5,25,4,it is too expensive,15,north,female,Detractors,Detractors,Others,True


### Well as you can see the syntax was able to idenitfy rows with 'price', 'high price','costly' and 'expensive'. Obviosly there are far more complex regex expressions that can be devised. but this is simple and effective method of EDA.
### This is the first of my cheetsheets. I will post some other cheetsheets in the coming days. Hope these were of use to you. Look forward to hearing from you in the comments section.